In [1]:
# ================================================
# 必要なライブラリのインポート
# ================================================

# --------------------------------
# 数値計算・データ処理
# --------------------------------
import numpy as np  # 数値計算ライブラリ（ベクトル・行列演算など）
import pandas as pd  # データフレーム操作（時系列の取り扱いに必須）

# --------------------------------
# 可視化ライブラリ
# --------------------------------
from matplotlib import pyplot as plt  # グラフ描画の基本ライブラリ
import matplotlib.dates as mdates  # 時系列軸のフォーマット設定に便利
import seaborn as sns  # 統計的な可視化スタイル（美しい出力）

sns.set()  # Seabornのデフォルトスタイルを有効化（見やすいグラフに）

# --------------------------------
# 統計モデリング・時系列分析
# --------------------------------
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.tsa.api as tsa  # 時系列専用ツール（自己相関, ARIMAなど）

# --------------------------------
# sktime：時系列処理の基本モジュール
# --------------------------------
from sktime.utils.plotting import plot_series  # 時系列データの簡単な可視化
from sktime.forecasting.base import ForecastingHorizon  # 予測対象期間の指定

# --------------------------------
# sktime：指数平滑法・ETSモデル（トレンド・季節性モデル）
# --------------------------------
from sktime.forecasting.exp_smoothing import ExponentialSmoothing  # Holt-Wintersモデル
from sktime.forecasting.ets import AutoETS  # 自動ETSモデル選択

# --------------------------------
# sktime：予測性能評価（誤差指標）
# --------------------------------
from sktime.performance_metrics.forecasting import (
    mean_absolute_scaled_error,  # MASE（基準モデルとの誤差比）
    MeanAbsoluteError,  # MAE（平均絶対誤差）
    mean_absolute_percentage_error,  # MAPE（平均絶対パーセント誤差）
    mean_absolute_error,  # MAE（関数形式）
)

# --------------------------------
# sktime：予測モデルの検証（クロスバリデーション等）
# --------------------------------
from sktime.forecasting.model_selection import (
    temporal_train_test_split,  # 時系列に基づく訓練・テスト分割
    ExpandingWindowSplitter,  # 時間窓を拡張しながらCVを実行
    ForecastingGridSearchCV,  # グリッドサーチによるハイパーパラメータ最適化
)
from sktime.forecasting.model_evaluation import evaluate  # CVの自動評価関数

# --------------------------------
# sktime：データ変換（前処理）
# --------------------------------
from sktime.transformations.series.boxcox import LogTransformer  # 対数変換など

# --------------------------------
# sktime：パイプライン・アンサンブル
# --------------------------------
from sktime.forecasting.compose import (
    TransformedTargetForecaster,  # 前処理＋予測モデルを一体化
    MultiplexForecaster,  # モデルの切替・比較
    EnsembleForecaster,  # モデルを統合して予測（単純平均など）
    AutoEnsembleForecaster,  # 自動アンサンブル（性能ベース加重など）
)
from sktime.transformations.compose import OptionalPassthrough  # 前処理を有無切替できる

# ================================================
# ✅ このブロックでできること
# - Holt-WintersやETSによる時系列予測
# - グリッドサーチ＋クロスバリデーション評価
# - 前処理を含むパイプライン設計
# - モデルアンサンブル・自動モデル選択
# ================================================

In [2]:
# 表示設定
np.set_printoptions(linewidth=60)
pd.set_option("display.width", 80)

from matplotlib.pylab import rcParams

rcParams["figure.figsize"] = 8, 4

In [3]:
# 飛行機乗客数データの読み込み
air_passengers = sm.datasets.get_rdataset("AirPassengers").data

# 日付インデックスの作成(PeriodIndex)
date_index = pd.period_range(start="1949-01", periods=len(air_passengers), freq="M")
air_passengers.index = date_index

# 不要な時間ラベルの削除
air_passengers = air_passengers.drop(air_passengers.columns[0], axis=1)

In [4]:
# データの分割
train, test = temporal_train_test_split(air_passengers, test_size=36)

# 予測期間
fh = np.arange(1, len(test) + 1)

In [5]:
data = pd.Series([1, 2, 3])
data

0    1
1    2
2    3
dtype: int64

In [ ]:
# ================================================
# 単純指数平滑法（Simple Exponential Smoothing）の準備
# ================================================

# 平滑化定数（α）
# 0 < α ≤ 1 の範囲で設定。
# αが大きいほど「新しいデータを重視」し、αが小さいほど「過去データを重視」する。
alpha = 0.8

# データの長さ分、(1 - α) を並べたリストを作成
# （指数平滑での過去データの重みを計算するために使用）
alpha_list = np.tile(1 - alpha, len(data))

# 結果の確認（例：0.2が3個並ぶなど）
alpha_list

array([0.2, 0.2, 0.2])

In [7]:
# --------------------------------
# 核心部分：重みの算出
# --------------------------------
#   α_list ** np.arange(0, len(data)) * α
#
# 具体的な意味：
#   np.arange(0, len(data)) → [0, 1, 2, 3, ...]
#   α_list ** np.arange(...) → [(1-α)^0, (1-α)^1, (1-α)^2, ...]
#   それに α を掛けることで：
#       [α*(1-α)^0, α*(1-α)^1, α*(1-α)^2, ...]
#   という指数的に減衰する重みが得られる。
#
# 例：α=0.8 の場合
#   → [0.8, 0.16, 0.032, 0.0064, ...] のように
#      最新ほど大きく、古いほど小さい重みになる。
weight = alpha_list ** np.arange(0, len(data)) * alpha

# 結果を確認
weight

array([0.8  , 0.16 , 0.032])

In [8]:
# ================================================
# 指数平滑法による予測値の計算
# ================================================

# 重みを逆順にする
# （理由）計算時に「古いデータ × 小さい重み」「新しいデータ × 大きい重み」
# が対応するように、重みの順序を入れ替える。
# これで最新の観測値ほど大きな重みが掛かるようになる。
weight = weight[::-1]

# --------------------------------
# 指数平滑化法による予測値の算出
# --------------------------------
# 予測値は、各データとその重みの積の総和で表される：
#
#   ŷ_t = Σ [ α * (1 - α)^(t - i) * y_i ]
#
# このコードでは「data * weight」で要素ごとの積をとり、
# np.sum() でそれを合計している。
#
# 例：
#   data = [y1, y2, y3, y4]
#   weight = [0.0064, 0.032, 0.16, 0.8]（α=0.8の場合）
#   → ŷ_4 = y1*0.0064 + y2*0.032 + y3*0.16 + y4*0.8
#
# したがって、新しい観測値ほど強く反映される。
print(f"4時点目の予測値 {np.sum(data * weight):.4g} ")

4時点目の予測値 2.752 


In [9]:
# データの並び順の変更
data_2 = pd.Series([3, 2, 1])

# 指数平滑化法による予測値
print(f"4時点目の予測値 {np.sum(data_2 * weight):.4g} ")

4時点目の予測値 1.216 


In [10]:
# 参考：単なるデータの平均
np.mean(data)

2.0

In [11]:
# ================================================
# 指数平滑法（単純指数平滑）の逐次計算
# ================================================

# 初期値（t=1 の平滑値）
# ------------------------------------------------
# 最初の平滑値 ŷ₁（ハット）は「予測の出発点」。
# 通常は観測値の初期値や0などを仮定する。
yhat1 = 0

# --------------------------------
# 各時点の予測値の更新式
# --------------------------------
# 一般式：
#     ŷ_t = α * y_{t-1} + (1 - α) * ŷ_{t-1}
#
# α（アルファ）は平滑化定数で、
#   ・α が大きい → 直近の観測値を重視（変化に敏感）
#   ・α が小さい → 過去の予測値を重視（変化に鈍感）

# 各時点での計算過程：
yhat2 = alpha * data[0] + (1 - alpha) * yhat1  # 2時点目の平滑値
yhat3 = alpha * data[1] + (1 - alpha) * yhat2  # 3時点目の平滑値
yhat4 = alpha * data[2] + (1 - alpha) * yhat3  # 4時点目の平滑値

# --------------------------------
# 結果の確認
# --------------------------------
# 各時点での予測値を表示する。
# 小数点4桁で整形して出力。
print(f"2時点目 {yhat2:.4g} | 3時点目 {yhat3:.4g} | 4時点目 {yhat4:.4g}")

# 💡 ポイント
# ・指数平滑法は再帰的に計算されるため、
#   逐次的に ŷ_t を更新していく構造を持つ。
# ・最新データが入るたびに予測値が滑らかに更新される。

2時点目 0.8 | 3時点目 1.76 | 4時点目 2.752


In [12]:
# 初期値
yhat1 = data[0]

# 予測値の計算
yhat2 = alpha * data[0] + (1 - alpha) * yhat1
yhat3 = alpha * data[1] + (1 - alpha) * yhat2
yhat4 = alpha * data[2] + (1 - alpha) * yhat3

# 結果の確認
print(f"2時点目 {yhat2:.4g} | 3時点目 {yhat3:.4g} | 4時点目 {yhat4:.4g}")

2時点目 1 | 3時点目 1.8 | 4時点目 2.76


In [13]:
# ================================================
# 単純指数平滑化法（Exponential Weighted Moving Average, EWMA）
# ================================================

# ewm(alpha=0.8, adjust=False)
# --------------------------------
# pandas.Series.ewm() は「指数加重移動平均」を計算するためのメソッドである。
# alpha（α）は平滑化定数を表す。
#   ・αが大きいほど、直近の観測値を重視（変化に敏感）
#   ・αが小さいほど、過去の値を重視（滑らかに変化）
#
# adjust=False の場合：
#   「再帰式による指数平滑法」と同じ計算を行う。
#   つまり、以下の式に従って逐次更新される：
#       ŷ_t = α * y_t + (1 - α) * ŷ_{t-1}

# adjust=True（デフォルト）の場合：
#   過去全体の重みを正規化して計算するため、
#   純粋な再帰形の指数平滑とは異なる（学術書での定義と異なる場合がある）。

# --------------------------------
# ewm() オブジェクトを生成
ewma_pd = data.ewm(alpha=0.8, adjust=False)

# --------------------------------
# mean() によって実際に平滑化を実行
# 各時点 t において、過去の観測値に指数的な重みを付けて平均を計算する。
ewma_pd.mean()

# 💡 計算イメージ
# t=1 から順に以下の再帰式で更新される：
#     ŷ₁ = y₁
#     ŷ_t = α * y_t + (1 - α) * ŷ_{t-1}   （t ≥ 2）
#
# これにより、最新の観測値に最も大きな重みが与えられ、
# 過去に遡るほど重みが指数的に減少していく。

0    1.00
1    1.80
2    2.76
dtype: float64

In [14]:
# ================================================
# 単純指数平滑法（Simple Exponential Smoothing）
# ================================================
# statsmodels.tsa.holtwinters.SimpleExpSmoothing は
# 「単純指数平滑法（Simple Exponential Smoothing）」を実装するクラスである。
#
# 基本式：
#   ŷ_t = α * y_t + (1 - α) * ŷ_{t-1}
#   （α：平滑化定数, 0 < α ≤ 1）
#
# 目的：
#   最新の観測値を重視しつつ、過去の観測値を指数的に減衰させて平均化する。
#   トレンドや季節性を持たない系列（定常系列）に有効。

# --------------------------------
# モデルの構築
# --------------------------------
ewma_sm = tsa.SimpleExpSmoothing(
    data,
    initialization_method="known",  # 初期化方法を指定
    initial_level=0,  # 初期値（ŷ₁）を明示的に 0 に設定
).fit(
    smoothing_level=0.8,  # 平滑化定数 α = 0.8
    optimized=False,  # α を自動最適化せず、手動で指定
)

# --------------------------------
# 当てはめ値（fittedvalues）の取得
# --------------------------------
# 各時点 t における指数平滑予測値 ŷ_t が格納されている。
# 再帰式：
#     ŷ₁ = 0  （初期値）
#     ŷ_t = 0.8 * y_t + 0.2 * ŷ_{t-1}
# として逐次的に更新されていく。
ewma_sm.fittedvalues

# 💡メモ：
#   initialization_method='known' により、初期レベルを明示的に指定可能。
#   これを 'heuristic' や 'estimated' にすると、自動的に初期値を推定してくれる。

0    0.00
1    0.80
2    1.76
dtype: float64

In [15]:
# ================================================
# 1期先の予測（forecast）
# ================================================
# SimpleExpSmoothing.fit() で得られたモデル ewma_sm を使って、
# 次の時点（t+1）の予測値を計算する。
#
# 単純指数平滑法における1期先予測の理論式：
#   ŷ_{t+1} = ŷ_t
#
# → すなわち、次の時点の予測値は「最新の平滑値」と等しい。
#   （未来に関する追加情報がないため、一定値を保つ形になる）

ewma_sm.forecast(1)

# 出力例：
# 4    2.4138
# dtype: float64
#
# ※これは「4時点目の予測値」を意味する。
#   3時点目までのデータをもとに、指数的に重み付けして求められた
#   最新の平滑値が次の期（t+1）の予測値となる。

3    2.752
dtype: float64

In [16]:
# ================================================
# 初期値を「初期データ（最初の観測値）」に設定した場合の指数平滑法
# ================================================

# initialization_method='legacy-heuristic' を指定すると、
# 初期平滑値（level_0）を「最初の観測値 y₁」として設定する。
# これは古典的な指数平滑法の実装方法であり、
# 多くの教科書で紹介される手法に対応する。

ewma_sm_lh = tsa.SimpleExpSmoothing(
    data, initialization_method="legacy-heuristic"  # 初期値 = 最初のデータ点
).fit(
    smoothing_level=0.8, optimized=False
)  # 平滑化定数 α=0.8 を手動指定

# 当てはめ値（fittedvalues）を確認
# ----------------------------------------
# 各時点における指数平滑化後の値（予測値）を出力する。
# この値は以下の漸化式で計算される：
#
#   ŷ_t = α·y_{t-1} + (1−α)·ŷ_{t−1}
#
# ただし、初期値として：
#   ŷ₁ = y₁
#
# つまり、最初の観測値が初期状態となるため、
# 初期値0のときよりも系列にスムーズに追従する結果となる。

ewma_sm_lh.fittedvalues

0    1.0
1    1.0
2    1.8
dtype: float64

In [17]:
# ================================================
# 1期先予測（単純指数平滑法）
# ================================================

# forecast(1) は、次の1時点先（t+1）の予測値を返す。
# 単純指数平滑法における1期先予測は以下の式で表される：
#
#   ŷ_{t+1} = ŷ_t
#
# つまり、「次の予測値」は「直前の平滑値」と等しい。
# これは、指数平滑法が“将来の変動を考慮しない”モデル
# （＝ランダムウォーク型の平滑予測）であることを意味する。

ewma_sm_lh.forecast(1)

3    2.76
dtype: float64

In [18]:
# ================================================
# 単純指数平滑法：初期値と平滑化定数の最適推定
# ================================================

# initialization_method='estimated' に設定することで、
# モデルが自動的に「初期値 (level_0)」と「平滑化定数 α（alpha）」を推定する。
# このとき、損失関数（通常は二乗誤差：Σ(y_t - ŷ_t)^2）を最小化するように
# パラメータが最適化される。

ewma_best = tsa.SimpleExpSmoothing(data, initialization_method="estimated").fit()

# ------------------------------------------------
# fittedvalues：訓練データ期間における当てはめ値
# ------------------------------------------------
# 各時点での予測値 ŷ_t を表す。
# これらは実測値 y_t に対して指数加重平均によって平滑化された値。

ewma_best.fittedvalues

0    1.0
1    1.0
2    2.0
dtype: float64

In [19]:
# ================================================
# 1期先予測（最適パラメータによる指数平滑法）
# ================================================

# forecast(1) は、「次の1時点先（t+1）」の予測値を返す。
# 単純指数平滑法では次の式で表される：
#
#   ŷ_{t+1} = α * y_t + (1 - α) * ŷ_t
#
# ただし、モデルが最適に推定した平滑化定数 α（alpha）と
# 初期値（level_0）を用いて算出される。
#
# 👉 この予測値は「直前の平滑値」と同じであり、
#    将来の予測は一定（＝水平な予測線）となるのが特徴。

ewma_best.forecast(1)

3    3.0
dtype: float64

In [20]:
# ================================================
# 推定されたパラメータの確認
# ================================================

# SimpleExpSmoothing.fit() の結果オブジェクトでは、
# 推定されたパラメータが `params` 属性に格納されている。
#
# 主な内容は以下の通り：
# --------------------------------
# smoothing_level : α（平滑化定数）
#   → 新しい観測値をどの程度重視するかを表すパラメータ。
#     値が1に近いほど最新のデータを強く反映する。
#
# initial_level   : 初期値（level_0）
#   → 平滑化を始める際の最初の予測値。
#     通常はデータの先頭値か、最適化によって推定される。
#
# SSE             : 残差平方和（誤差の合計）
#   → モデル当てはめ時の誤差の総量を表す。
# --------------------------------

ewma_best.params

{'smoothing_level': 0.9999999850988374,
 'smoothing_trend': nan,
 'smoothing_seasonal': nan,
 'damping_trend': nan,
 'initial_level': 1.0000000088092982,
 'initial_trend': nan,
 'initial_seasons': array([], dtype=float64),
 'use_boxcox': False,
 'lamda': None,
 'remove_bias': False}